In [1]:
import numpy as np
import pandas as pd
import utm
import math
import pickle
import os
import sys
sys.path.append('./src')
import time
import datetime
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import copy

import display_gm
import display_osm
import plot
import tools
import hmm, uni_hmm, split
import celltower as ctcrawler
from data_cleaning import noise, load, counter

# Load Data

In [2]:
load = reload(load)
file_2g = './data/jiading/data_2g.csv'
gongcan_2g = './data/jiading/2g_gongcan.csv'
final_gongcan_2g = './data/jiading/final_2g_gongcan.csv'

In [177]:
db, db_gps, towers = load.load_2g(file_2g, gongcan_2g, merge_tower=False, neighbor=False, with_rssi=False, radio_angle=False, context=False)
print 'len(db):', len(db), 'len(towers):', len(towers)

Unique Cell Tower:  81
Duplicated cell towers: [[66, 67, 68], [35, 36], [53, 54], [69, 70, 71], [40, 41], [56, 57], [61, 62, 63]]
Totally duplicate: 86
len(db): 72 len(towers): 91


In [178]:
db_gps[0][:5]

[(31.29184524, 121.21368387, 62, 0.0, 1507627341L),
 (31.28926598, 121.21361016, 62, 0.68, 1507627346L),
 (31.28922812, 121.21357512, 63, 0.0, 1507627351L),
 (31.28926254, 121.21358264, 63, 0.0, 1507627356L),
 (31.28928476, 121.21360502, 63, 0.0, 1507627359L)]

# Data Cleaning

In [179]:
noise = reload(noise)
dbs = noise.clean_db(db, max_dist=100, min_dist=100, min_len=10, max_again=10, debug=True)
dbs_gps = noise.clean_db_gps(db_gps, max_dist=100, min_dist=100, min_len=10, max_again=10, debug=False)
len(dbs), len(dbs_gps)

Traj ID=0
0->1:286m	
discard[0:0]=0m	keep[1:24]as id=0	
Traj ID=1
7->8:510m	
discard[0:7]=0m	keep[9:157]as id=1	
Traj ID=2
1->2:330m	
discard[0:1]=0m	keep[3:77]as id=2	
Traj ID=3

keep[1:33]as id=3	
Traj ID=4
0->1:394m	
discard[0:0]=0m	keep[2:44]as id=4	
Traj ID=5
1->2:118m	
discard[0:1]=0m	keep[3:48]as id=5	
Traj ID=6
1->2:522m	112->113:752m	133->134:116m	
discard[0:1]=0m	trimed[0,89] keep[2:23]as id=6	discard[113:133]=33m	discard[134:135]=0m	
Traj ID=7
9->10:101m	
discard[0:9]=0m	keep[10:112]as id=7	
Traj ID=8

keep[0:373]as id=8	
Traj ID=9
1->2:947m	
discard[0:1]=0m	keep[2:341]as id=9	
Traj ID=10

keep[0:420]as id=10	
Traj ID=11

keep[1:295]as id=11	
Traj ID=12

keep[1:422]as id=12	
Traj ID=13
35->36:838m	
discard[0:35]=0m	keep[37:1015]as id=13	
Traj ID=14

keep[1:826]as id=14	
Traj ID=15
42->43:667m	
keep[5:42]as id=15	keep[43:114]as id=16	
Traj ID=16
2->3:482m	345->346:714m	
discard[0:2]=0m	keep[6:345]as id=17	keep[346:853]as id=18	
Traj ID=17

trimed[1,12] keep[1:234]as id=19	
Tr

(74, 74)

In [180]:
db_gps[0][:5]

[(31.29184524, 121.21368387, 62, 0.0, 1507627341L),
 (31.28926598, 121.21361016, 62, 0.68, 1507627346L),
 (31.28922812, 121.21357512, 63, 0.0, 1507627351L),
 (31.28926254, 121.21358264, 63, 0.0, 1507627356L),
 (31.28928476, 121.21360502, 63, 0.0, 1507627359L)]

# Ground Truth

In [181]:
load = reload(load)
noise = reload(noise)
match_res, time_set = load.load_matching('./data/jiading/matching_out', len(dbs), 50)
dbs = noise.reclean(dbs, time_set, debug=True)
dbs_gps = noise.reclean(dbs_gps, time_set)

traj id= 1 150 140
traj id= 13 980 976
traj id= 22 190 167
traj id= 23 216 198
traj id= 25 427 420
traj id= 26 185 149
traj id= 41 181 173
traj id= 44 136 131
traj id= 50 219 218
traj id= 51 322 316
traj id= 63 34 32
traj id= 67 89 87
traj id= 72 50 44


# Split Data

In [184]:
split = reload(split)
# train:(origin idx, origin feature)
# train, test = hmm.k_split(dbs, k=4)
train, test, gps3 = split.k_g_split(dbs, k=4, gpsize=0.0)
# train, test = hmm.hand_split(dbs, k=2)
# len(gps3)

In [185]:
train_set = set()
for tr_id, traj in train.iteritems():
    for point in traj:
        train_set.add(point[1][2:-2])
test_set = set()
for tr_id, traj in test.iteritems():
    for point in traj:
        test_set.add(point[1][2:-2])
test_set-train_set

set()

# Layer 1

## random forest

In [209]:
from sklearn.ensemble import RandomForestClassifier
def re_organize(db):
    X, y = [], []
    range_dict = dict()
    start_idx = 0
    for tr_id, traj in db.iteritems():
        piece_match = match_res[tr_id]
        for old_idx, point in traj:
            obsv = point[2:-2]
            rid = piece_match[old_idx][3]
            X.append(obsv)
            y.append(rid)
        range_dict[tr_id] = (start_idx, len(X))
        start_idx = len(X)
    return X, y, range_dict
X_train, Y_train, _ = re_organize(train)
X_test, Y_test, range_dict = re_organize(new_test)

In [210]:
clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [211]:
Y_predict = clf.predict_proba(X_test)

In [212]:
help_info = dict()
min_len = 10
for tr_id in range_dict.iterkeys():
    s_idx, e_idx = range_dict[tr_id]
    help_info[tr_id] = []
    for i, piece in enumerate(Y_predict[s_idx:e_idx]):
        length = len(set(piece)) - 1
        k = min(min_len, length)
        idxes = sorted(range(len(piece)), key=lambda x: piece[x], reverse=True)[:k]
        prob_info = dict()
        for idx in idxes:
            prob_info[clf.classes_[idx]] = piece[idx]
        help_info[tr_id].append(prob_info)

In [174]:
summary = dict()
for min_len in range(1, 21):
    cnt = 0
    # statist = []
    for i, piece in enumerate(Y_predict):
        length = len(set(piece)) - 1
        k = min(min_len, length)
        idxes = sorted(range(len(piece)), key=lambda x: piece[x], reverse=True)[:k]
        rids = [clf.classes_[idx] for idx in idxes]
        if Y_test[i] in rids:
    #         statist.append((i, rids.index(Y_test[i]), length))
            cnt += 1
    summary[min_len] = cnt * 1.0 / len(Y_test)

In [ ]:
plt.figure(figsize=(20, 10))
# plt.plot(one_ids.keys(), one_ids.values())
# plt.plot(two_ids.keys(), two_ids.values())
plt.plot(context_ids.keys(), context_ids.values())
xticks = (str(x) for x in summary.keys())
plt.xticks(summary.keys(), xticks)
# plt.legend(['ID1', 'ID1ID2', 'Context_ID1'], loc='best')
plt.title('RF predict_proba result')
plt.xlabel('min candidate')
plt.ylabel('predict_proba result')
plt.show()
# plt.savefig('./display/deepin/RF_predict_proba.png')

In [110]:
pd.DataFrame(statist, columns=['i', 'index', 'length']).describe()

,i,index,length
count,2775.000000,2775.000000,2775.000000
mean,1508.603964,0.405405,4.777658
std,876.029164,0.653568,3.555402
min,2.000000,0.000000,1.000000
25%,762.500000,0.000000,2.000000
50%,1489.000000,0.000000,4.000000
75%,2221.500000,1.000000,7.000000
max,3087.000000,2.000000,26.000000


# Layer 2

In [237]:
uni_hmm = reload(uni_hmm)

In [238]:
bounding_box = (328000, 331000, 3462000, 3465000)
side = 30
sag = uni_hmm.Sag(side, bounding_box)

In [239]:
A, cell_db = sag.get_trans_mat(match_res, gps3.keys())
uni_cells = sag.get_cells(cell_db)
len(uni_cells)

0

In [240]:
B, D = sag.get_obsv_mat(train, match_res, version=0)

In [241]:
# clean test dataset
noise = reload(noise)
new_test = noise.clean_testset(sag, test, match_res, B, debug=True)

TrajID=0 Before=6 End=5
TrajID=1 Before=35 End=30
TrajID=3 Before=8 End=7
TrajID=4 Before=11 End=10
TrajID=5 Before=12 End=10
TrajID=7 Before=26 End=24
TrajID=8 Before=94 End=81
TrajID=9 Before=85 End=82
TrajID=10 Before=105 End=102
TrajID=11 Before=74 End=67
TrajID=12 Before=106 End=101
TrajID=13 Before=244 End=242
TrajID=14 Before=207 End=203
TrajID=19 Before=59 End=57
TrajID=20 Before=62 End=61
TrajID=28 Before=95 End=94
TrajID=30 Before=21 End=20
TrajID=36 Before=27 End=24
TrajID=39 Before=28 End=26
TrajID=40 Before=22 End=19
TrajID=41 Before=43 End=42
TrajID=43 Before=25 End=23
TrajID=44 Before=33 End=29
TrajID=52 Before=27 End=26
TrajID=65 Before=122 End=119
TrajID=71 Before=17 End=15


In [235]:
tr_id = 1
traj = new_test[tr_id]
help_list = help_info[tr_id]
speeds = [point[1][-2] for point in traj]
k = math.floor(np.std(speeds)) + 1
self = sag
debug = False
f, p = [], defaultdict(dict)
h = dict()
obsv = self.get_obsv(traj[0])
help_dict = help_list[0]
for state, prob in B[obsv].iteritems():
    rid = self.get_rid(state)
    cid = self.get_cid(state)
    if help_dict.has_key(rid):
        h[cid] = prob * help_dict[rid]
f.append(h)
for idx in range(1, len(traj)):
    help_dict = help_list[idx]
    obsv = self.get_obsv(traj[idx])
    speed, timestp = traj[idx][1][-2:]
    h = dict()
    _obsv = self.get_obsv(traj[idx-1])
    _speed, _timestp = traj[idx-1][1][-2:]
    max_dist = max(_speed, speed) * (timestp - _timestp) + 1.0 * self.side
    min_dist = min(_speed, speed) * (timestp - _timestp) - 1.0 * self.side
    for state, prob in B[obsv].iteritems():
        max_p = 0
        rid = self.get_rid(state)
        cid = self.get_cid(state)
        if not help_dict.has_key(rid):
            continue
        x1, y1 = self.cell2index(cid)
        for _state in B[_obsv].iterkeys():
            _rid = self.get_rid(state)
            _cid = self.get_cid(state)
            if not f[idx-1].has_key(_cid):
                continue
            x2, y2 = self.cell2index(_cid)
            avg_act_dist = (math.sqrt((x2-x1)**2 + (y2-y1)**2)) * self.side
            t1 = t2 = t3 = 0
            t1 = f[idx-1][_cid]
            t3 = B[obsv][state]
            if not (min_dist - self.side * k <= avg_act_dist <= max_dist + self.side * k):
                if debug:
                    print 'idx=%d,(%d,%d)'%(idx, _cid, cid)
                    print 'dist(%d,%d) act(%d)' % (int(min_dist), int(max_dist), int(avg_act_dist))
                continue
            t2 = self.transfer(_cid, cid)
            if avg_act_dist < min_dist:
                t2 *= math.e ** (-(avg_act_dist - min_dist)**2/(min_dist**2)/2)
            if avg_act_dist > max_dist:
                t2 *= math.e ** (-(avg_act_dist - max_dist)**2/(max_dist**2)/2)
            alt_p = t1 + t2 * t3
            if alt_p > max_p:
                max_p = alt_p
                p[idx][cid] = _cid
            h[cid] = max_p
    f.append(h)
    if debug:
        print idx
sl = []
max_prob = max(f[-1].itervalues())
last_s = max(f[-1].iterkeys(), key=lambda k: f[-1][k])
for idx in range(len(traj)-1, 0, -1):
    sl.append(last_s)
    last_s = p[idx][last_s]
sl.append(last_s)
sl.reverse()

ValueError: max() arg is an empty sequence

In [232]:
sl, len(traj)

([4767, 4767, 4767, 4767, 4767], 5)

In [242]:
cell_pred = dict()
statistic = []
failed_trajs = []
for tr_id, traj in new_test.iteritems():
    # 提升6m左右精度
    speeds = [point[1][-2] for point in traj]
    k = math.floor(np.std(speeds))
#     k = 1
    try:
        sl, max_prob= sag.viterbi_2(A, B, traj, k, help_info[tr_id])
    except:
        failed_trajs.append(tr_id)
        continue
#     prob = sag.backforward(A, B, traj, match_res[tr_id], k, debug=False)
    prob = 0
    statistic.append((tr_id, max_prob, prob))
    cell_pred[tr_id] = sl
    print 'TrajID=' + str(tr_id)
print 'failed:', failed_trajs, len(failed_trajs)

TrajID=0
TrajID=2
TrajID=15
TrajID=16
TrajID=23
TrajID=31
TrajID=32
TrajID=33
TrajID=50
failed: [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73] 65


In [243]:
uni_prec, uni_prec_dict = sag.evaluate(new_test, cell_pred, match_res)

In [244]:
precs = [(tr_id, prec) for tr_id, prec in uni_prec_dict.iteritems()]
df_prob = pd.DataFrame(statistic, columns=['TrajID', 'MaxProb', 'RealProb'])
df_prec = pd.DataFrame(precs, columns=['TrajID', 'Precision'])

In [245]:
merged = pd.merge(df_prob, df_prec).sort_values(by=['Precision'], ascending=False).reset_index(drop=True)
error_ids = list(merged[merged['Precision'] > 100]['TrajID'].values)
merged

,TrajID,MaxProb,RealProb,Precision
0,15,10.541345,0,633.434498
1,0,3.809279,0,383.889664
2,33,17.519726,0,287.639737
3,50,2.945119,0,220.828086
4,16,7.400856,0,198.012069
5,23,15.213533,0,182.416851
6,32,10.117870,0,180.689188
7,31,14.521126,0,111.287973
8,2,17.308975,0,67.524324


In [246]:
np.median(uni_prec), np.median(uni_prec_dict.values()), np.mean(uni_prec), len(uni_prec_dict)

(193.67369897917462, 198.01206873568023, 218.66813120165656, 9)

## HMM

In [88]:
hmm = reload(hmm)

In [72]:
A = hmm.get_trans_mat(match_res)

transition 12425 times


In [73]:
B, D = hmm.get_obsv_mat(train, match_res)

In [75]:
# clean test dataset
new_test = dict()
for tr_id, traj in test.iteritems():
    new_test[tr_id] = []
    matched = match_res[tr_id]
    new_points = []
    for point in traj:
        obsv = point[1][2:-2]
        state = matched[point[0]][3]
        if B.has_key(obsv) and B[obsv].has_key(state):
            new_points.append(point)
    new_test[tr_id] = new_points
    if len(new_points) != len(traj):
        print 'TrajID=%d, Before/After=%d/%d' % (tr_id, len(traj), len(new_points))

TrajID=0, Before/After=6/5
TrajID=7, Before/After=25/24
TrajID=8, Before/After=93/92
TrajID=9, Before/After=85/83
TrajID=13, Before/After=244/243
TrajID=15, Before/After=11/10
TrajID=18, Before/After=127/126
TrajID=20, Before/After=62/61
TrajID=28, Before/After=95/93
TrajID=29, Before/After=122/121
TrajID=30, Before/After=21/20
TrajID=35, Before/After=20/18
TrajID=37, Before/After=30/29
TrajID=41, Before/After=43/41
TrajID=44, Before/After=32/31
TrajID=50, Before/After=54/52
TrajID=51, Before/After=79/77
TrajID=54, Before/After=13/12
TrajID=55, Before/After=18/17
TrajID=57, Before/After=13/12
TrajID=59, Before/After=12/11
TrajID=65, Before/After=121/120
TrajID=68, Before/After=23/22
TrajID=69, Before/After=20/18
TrajID=72, Before/After=11/10


In [85]:
statistic = []
failed_ids = []
road_pred = dict()
for tr_id, traj in new_test.iteritems():
    sl, max_prob, real_prob = [], 0, 0
    try:
        sl, max_prob = hmm.viterbi(A, B, traj)
#         real_prob = hmm.backforward(A, B, traj, match_res[tr_id], debug=False)
        statistic.append((tr_id, max_prob, real_prob))
        road_pred[tr_id] = sl
    except:
        failed_ids.append(tr_id)
        continue
    print 'TrajID=' + str(tr_id)
failed_ids

TrajID=0
TrajID=1
TrajID=2
TrajID=3
TrajID=4
TrajID=5
TrajID=6
TrajID=7
TrajID=8
TrajID=9
TrajID=10
TrajID=11
TrajID=12
TrajID=13
TrajID=14
TrajID=15
TrajID=16
TrajID=17
TrajID=18
TrajID=19
TrajID=20
TrajID=21
TrajID=22
TrajID=23
TrajID=24
TrajID=25
TrajID=26
TrajID=27
TrajID=28
TrajID=29
TrajID=30
TrajID=31
TrajID=32
TrajID=33
TrajID=34
TrajID=35
TrajID=36
TrajID=37
TrajID=38
TrajID=39
TrajID=40
TrajID=41
TrajID=42
TrajID=43
TrajID=44
TrajID=45
TrajID=46
TrajID=47
TrajID=48
TrajID=49
TrajID=50
TrajID=51
TrajID=52
TrajID=53
TrajID=54
TrajID=55
TrajID=56
TrajID=57
TrajID=58
TrajID=59
TrajID=60
TrajID=61
TrajID=62
TrajID=63
TrajID=64
TrajID=65
TrajID=66
TrajID=67
TrajID=68
TrajID=69
TrajID=70
TrajID=71
TrajID=72
TrajID=73


[]

In [89]:
pred = hmm.evaluate(new_test, road_pred, match_res)

In [91]:
np.median(pred.values())

0.5239651416122004

In [93]:
tr_id = 1
traj = new_test[tr_id]
matched = match_res[tr_id]
predict = road_pred[tr_id]
cnt = 0
comp = []
for old_idx, point in traj:
    rid = matched[old_idx][3]
    comp.append((rid, predict[cnt]))
pd.DataFrame(comp, columns=['RealRid', 'PredRid'])

,RealRid,PredRid
0,164316039,164316039
1,164316039,164316039
2,164316039,164316039
3,164316039,164316039
4,164316039,164316039
5,164316039,164316039
6,164316039,164316039
7,164316039,164316039
8,164316039,164316039
9,164316039,164316039


In [86]:
df_prob = pd.DataFrame(statistic, columns=['TrajID', 'MaxProb', 'RealProb'])
df_prob

,TrajID,MaxProb,RealProb
0,0,1.962963,0
1,1,2.049272,0
2,2,18.581395,0
3,3,0.715420,0
4,4,0.387875,0
5,5,1.213731,0
6,6,0.096279,0
7,7,1.032155,0
8,8,9.782088,0
9,9,1.107590,0


# Display

In [51]:
sum1, sum2 = dict(), dict()
r_rid = 0
for rid in D.iterkeys():
    sum1[r_rid] = len(D[rid])
    r_rid += 1
for obsv in B.iterkeys():
    num = len(B[obsv])
    if not sum2.has_key(num):
        sum2[num] = 0
    sum2[num] += 1

In [57]:
plot = reload(plot)

In [50]:
len(B)

437

In [58]:
plot.draw_barplot(sum2, 'n(obsv) of n(road)', 'n(road)', 'n(obsv)')